# Preparing for batching
Now the basic structure of the spiking neural networks has been made it is time to tidy the code and prepare it for batch runs on the Goliath cluster. The next cell will form a support file for reading and graphing the data.

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')  # must be done on cluster as there is no display
from brian2 import *
import struct
import os

V3 = "aedat3"
V2 = "aedat"  # current 32bit file format
V1 = "dat"  # old format

EVT_DVS = 0  # DVS event type
EVT_APS = 1  # APS event

RESX = 128
RESY = 128
RES = RESX * RESY
COLOURS = ['r', 'g', 'b', 'm', 'c', 'm', 'y', 'k']

##      Some helpful functions
flatIndex = lambda x, y : y*RESX + x
squareIndex = lambda i : (i % 128, int(i / 128))  # TODO there is probably some math todo for DAVIS 



def loadaerdat(datafile='/tmp/aerout.dat', length=0, version=V2, debug=1, camera='DVS128'):
    """    
    load AER data file and parse these properties of AE events:
    - timestamps (in us), 
    - x,y-position [0..127]
    - polarity (0/1)

    @param datafile - path to the file to read
    @param length - how many bytes(B) should be read; default 0=whole file
    @param version - which file format version is used: "aedat" = v2, "dat" = v1 (old)
    @param debug - 0 = silent, 1 (default) = print summary, >=2 = print all debug
    @param camera='DVS128' or 'DAVIS240'
    @return (ts, xpos, ypos, pol) 4-tuple of lists containing data of all events;
    """
    # constants
    aeLen = 8  # 1 AE event takes 8 bytes
    readMode = '>II'  # struct.unpack(), 2x ulong, 4B+4B
    td = 0.000001  # timestep is 1us   
    if(camera == 'DVS128'):
        xmask = 0x00fe
        xshift = 1
        ymask = 0x7f00
        yshift = 8
        pmask = 0x1
        pshift = 0
    elif(camera == 'DAVIS240'):  # values take from scripts/matlab/getDVS*.m
        xmask = 0x003ff000
        xshift = 12
        ymask = 0x7fc00000
        yshift = 22
        pmask = 0x800
        pshift = 11
        eventtypeshift = 31
    else:
        raise ValueError("Unsupported camera: %s" % (camera))

    if (version == V1):
        print ("using the old .dat format")
        aeLen = 6
        readMode = '>HI'  # ushot, ulong = 2B+4B

    aerdatafh = open(datafile, 'rb')
    k = 0  # line number
    p = 0  # pointer, position on bytes
    statinfo = os.stat(datafile)
    if length == 0:
        length = statinfo.st_size    
    print ("file size", length)
    
    # header
    lt = aerdatafh.readline().decode('utf-8')
    while lt and lt[0] == "#":
        p += len(lt)
        k += 1
        lt = aerdatafh.readline().decode('utf-8') 
        if lt.startswith('#End of Preferences'):
            p += len(lt)   # TODO this is a bit lazy, rearrange logic
            k += 1
            break
        if debug >= 2:
            #print (str(lt))
            pass
        continue
    
    # variables to parse
    timestamps = []
    xaddr = []
    yaddr = []
    pol = []
    
    # read data-part of file
    aerdatafh.seek(p)
    s = aerdatafh.read(aeLen)
    p += aeLen
    length += p
    
    #print (xmask, xshift, ymask, yshift, pmask, pshift, aeLen,  p, length)    
    while p < length:
        #print (xmask, xshift, ymask, yshift, pmask, pshift, aeLen,  p, length)
        #ii = int.from_bytes(s, byteorder='big')
        #print(str(bin(ii)), len(str(bin(ii))))
        #print(type(s))
        addr, ts = struct.unpack(readMode, s)
        
        # parse event type
        if(camera == 'DAVIS240'):
            eventtype = (addr >> eventtypeshift)
        else:  # DVS128
            eventtype = EVT_DVS
        
        # parse event's data
        if(eventtype == EVT_DVS):  # this is a DVS event
            #print(str(bin(addr)), len(str(bin(addr))))
            x_addr = (addr & xmask) >> xshift
            y_addr = (addr & ymask) >> yshift
            a_pol = (addr & pmask) >> pshift


            if debug >= 3: 
                print("ts->", ts)  # ok
                print("x-> ", x_addr)
                print("y-> ", y_addr)
                print("pol->", a_pol)

            timestamps.append(ts)
            xaddr.append(x_addr)
            yaddr.append(y_addr)
            pol.append(a_pol)
            #poldf
        
        aerdatafh.seek(p)
        s = aerdatafh.read(aeLen)
        p += aeLen        

    if debug > 0:
        print(len(timestamps))
        try:
            print ("read %i (~ %.2fM) AE events, duration= %.2fs" % (len(timestamps), len(timestamps) / float(10 ** 6), (timestamps[-1] - timestamps[0]) * td))
            n = 5
            print ("showing first %i:" % (n))
            print ("timestamps: %s \nX-addr: %s\nY-addr: %s\npolarity: %s" % (timestamps[0:n], xaddr[0:n], yaddr[0:n], pol[0:n]))
        except Exception as e:
            print(e)
            print ("failed to print statistics")

    return timestamps, xaddr, yaddr, pol


def rec_field(ofx, ofy, sizex, sizey, stim_res_x=RESX, stim_res_y=RESY):
    """ Given the offset in (x,y) and a size in x and y return a receptive field
        as a list of indices assuming camera stimulus if not specified for stim_res_*
        The offset plus corresponding size will not excede the retina
    """
    if ofx + sizex >= stim_res_x:
        sizex = stim_res_x - ofx
    if ofy + sizey >= stim_res_y:
        sizey = stim_res_y - ofy
    return [flatIndex(x, y) for x in range(ofx, sizex+ofx) for y in range(ofy, sizey+ofy)]

def dvs2group(xs, ys, ts):
    """ Given DVS data convert it to indices and timestamps that can be used by a Brian2 Spiking
        Neuron group. 
    """
    # TODO This means no 2 distinct neurons can fire at the same us.... Needs a fix
    real_indices = [flatIndex(*xy) for xy in zip(xs, ys)]
    real_times, uniq_idxs = np.unique(ts, return_index=True)
    real_indices = np.asarray(real_indices)[uniq_idxs]
    real_times = (real_times - real_times[0])*us
    return (real_indices, real_times)

 


In [ ]:
##   SOMEWHAT CONSTANTS
defaultclock.dt = 1*us
tau = 10*ms
eqs = '''
dv/dt = (I - v) / tau : 1 (unless refractory)  # Pick any model
I = 0: 1
'''
## LOAD REAL DATA
file = 'M1a_expand.aedat'
bytes2read = 30000
# Finally load and process the data once
ts, xs, ys, ps = loadaerdat(datafile=file, length=bytes2read, version='aedat', debug=0, camera='DVS128')
real_indices, real_times = dvs2group(xs, ys, ts)

Now defining the actual network 

In [ ]:
def main():
    field_size=32
    stride=16
    dist = lambda p1, p2: np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    for field_size in [32, 16]:

        field_sqrd = field_size * field_size
        surfs = []
        surf_names = 'CE'#'CELRDU'
        ##                              Add expansions
        mid = (field_size - 1) / 2 
        max_dist = dist((0,0), (mid, mid))
        surfs.append(np.array([ dist((i%field_size, i // field_size), (mid, mid)) for i in range(field_sqrd)])/max_dist) # Contraction
        surfs.append(1 - surfs[-1])                                                                   # Expansion
        ##                              Add horizontal and vertical
        #surfs.append(np.array([(i%field_size)/field_size for i in range(field_sqrd)]))                #left active
        #surfs.append(np.array([(field_size - (i%field_size))/field_size for i in range(field_sqrd)])) #right active
        #surfs.append(1 - surfs[-1])
        #surfs.append(np.array([(i//field_size)/field_size for i in range(field_sqrd)]))         #down active
        #surfs.append(np.array([1 - (i//field_size)/field_size for i in range(field_sqrd)]))       #up active
        #surfs.append(1 - surfs[-1])

        for stride in [16, 8]:
            for s_num in range(len(surfs)):
                surface = surfs[s_num]
                #print(surf.shape)
                name = '{}_{}field_{}stride.png'.format(surf_names[s_num], field_size, stride)
                voltMon, spikeMon_opt, locs, rcsizes = make_network(surface, field_size, stride)           # Make network
                graph_network(voltMon, spikeMon_opt, locs, rcsizes, surface, save=name)                    # Graph network



def make_network(surface, field_size=32, stride=8):
    start_scope()

    # build receptive fields
    field_sqrd = field_size * field_size
    inp = SpikeGeneratorGroup(RES, real_indices, real_times)

    G, S, locs = conv_layer(inp, (RESY, RESY), surface, (field_size, field_size), stride)
    rcsizes = [(field_size, field_size) for i in range(len(locs))]
    G.v = 0.1

    voltMon = StateMonitor(G, 'v', record=True)
    spikeMon_opt = SpikeMonitor(G)
    
    run(60*ms)

    return (voltMon, spikeMon_opt, locs, rcsizes)


def graph_network(voltMon, spikeMon_opt, locs, rcsizes, surface, save=False):
    figure(figsize=(14, 12))
    suptitle('{} #neurons: '.format(save) + str(len(locs)))
    subplot(221)
    for n in range(len(locs)):
        plot(voltMon.t/ms, voltMon.v[n], '-'+COLOURS[n%len(COLOURS)], lw=1, label='N'+str(n))

    axhline(0.7, ls=':', c='g', lw=3)  # Threshold line
    xlabel('Time (ms)')
    ylabel('v')
    title('Membrane potentials')

    #legend(loc='best')

    # Plot image and RC locations
    ax = subplot(222)
    plot(xs, ys, '.k')
    for n in range(len(locs)):
        ax.add_patch(Rectangle(locs[n], *(rcsizes[n]), lw=3, color=COLOURS[n%len(COLOURS)], fill=False))
    title('Receptive fields of each kernel')

    subplot(223)
    vis_surface(surface, field_size, field_size)
    title('Convolution kernel')

    subplot(224)
    title('Number of output spikes from each neuron in second layer')
    vis_output(spikeMon_opt, np.sqrt(len(locs)), np.sqrt(len(locs)))
    
    if save:
        savefig(save, bbox_inches='tight')

def vis_output(spikeMon, resx, resy):
    """ Given a spikeMonitor with variables t and i identifying neurons convert
        back to 2D coordinates and plot the response
    """
    times = spikeMon.t
    indexs_2dx = [i % resx for i in spikeMon.i]
    indexs_2dy = [int(i / resy) for i in spikeMon.i]
    
    heatmap, xedges, yedges = np.histogram2d(indexs_2dx, indexs_2dy, bins=resx)
    extent = [0,resx, 0,resy]#[xedges[0], xedges[-1], yedges[0], yedges[-1]]

    imshow(heatmap.T, extent=extent, origin='lower', interpolation='None')
    colorbar()

def vis_surface(surface, sizex=None, sizey=None):
    """ Given a surface draw it as a heatmap. If size not supplied, render in current
        shape, otherwise reshape to sizes given. Both arguements required or
        no reshaping will happen.
    """
    res = surface
    if sizex != None and sizey != None:
        res = np.zeros(shape=(sizey, sizex))
        for i in range(sizey):
            res[i, :] = surface[i*sizex : (i+1)*sizex]
            
    imshow(res, interpolation='None')
    colorbar()
    
def conv_layer(layer, layer_rc, kernel, kernel_rc, stride):
    """ Given a neuron layer and a surface (kernel of weights), create a set of synapses
        and a new layer that convolves that surface with the neuron layer with stride stride. 
        Returns tuple of (new_layer, new_synapses, locations)

        layer - a (NeuronGroup) group of neurons being input to new layer
        layer_rc - number of rows and colums of layer as tuple (row, col)
        kernel - 1D array of weights to be used as convolution kernel
        kernel_rc - rows and cols of kernel
        stride - stride between kernel convolutions
        locations - location of each receptive field
    """
    kr, kc = kernel_rc
    lr, lc = layer_rc
    
    r, c = (0, 0)
    locs = []  # Compute bottom left corner of each position for kernel
    while c + kc <= lc:    # TODO this can be faster if done analytically... 
        r = 0
        while r + kr <= lr:
            locs.append((r,c))
            r += stride
        c += stride
            
    print("Loc Num: ", len(locs))        
    ng = NeuronGroup(len(locs), eqs, threshold='v>0.7', reset='v=0.1', method='linear', refractory=5*ms)
    syns = Synapses(layer, ng, 'w : 1', on_pre='v_post += w')
    
    for n in range(len(locs)):
        arr = np.array(rec_field(*(locs[n]), kr, kc))
        syns.connect(i=arr, j=n)
        syns.w[:, n] = surface*0.05
    
    return (ng, syns, locs)
    
if __name__ == '__main__':
    main()